In [ ]:
# %% md
# # 03  Sensitivity Analysis
# Heat-maps over branching ratio & resilience speed
# %%
import numpy as np, pandas as pd, matplotlib.pyplot as plt, os
os.makedirs("../docs/figures", exist_ok=True)
# %% 1. parameter grid
branching = np.linspace(0.2, 0.8, 7)   # alpha/beta
resilience = np.linspace(1/60, 1/10, 6)  # rho
T, x0 = 3_600, 100_000
# %% 2. helpers
import sys; sys.path.append("..")
from sim.simulate_hawkes import simulate_bivariate_hawkes
from sim.simulate_orderflow import simulate_orderbook_sequence
from backtest.backtester import run_backtest, hawkes_lq_strategy
from models.hawkes_control import compute_feedback_gains

def cost_for_params(br, rho):
    # Hawkes params
    alpha = br; beta = 1.0
    mu_vec = np.array([0.5, 0.5])
    A = np.array([[alpha, alpha*0.7],[alpha*0.7, alpha]])
    betas = np.array([[beta, beta],[beta, beta]])
    trace = simulate_orderbook_sequence(
        T, dict(mu_vec=mu_vec, A=A, betas=betas),
        dict(permanent=0.0, transient=dict(alphas=[5e-4,2e-4], betas=[10,50]), rho=rho),
        dict(name='fixed', size=10), initial_price=100, dt_ms=1, seed=42)
    # LQ gains
    gains = compute_feedback_gains(
        np.array([5e-4,2e-4]), np.array([10.,50.]), rho,
        np.array([alpha, alpha*0.7]), np.array([beta, beta]), A,
        1e-4, 5e-5, 1e-4, T, 120)
    res = run_backtest(trace, hawkes_lq_strategy,
                       dict(gains=gains,
                            propagator_alphas=np.array([5e-4,2e-4]),
                            propagator_betas=np.array([10.,50.]),
                            propagator_rho=rho,
                            hawkes_betas=np.array([beta, beta]),
                            hawkes_A=A,
                            eta=1e-4, gamma=5e-5),
                       x0, T)
    return res['costs']['total_cost']
# %% 3. sweep
records = []
for br in branching:
    for rho in resilience:
        records.append({'branching': br, 'rho': rho,
                        'cost': cost_for_params(br, rho)})
sens = pd.DataFrame(records)
# %% 4. heat-map
piv = sens.pivot(index='branching', columns='rho', values='cost')
plt.figure(figsize=(4,3))
sns = plt.imshow(piv.values, aspect='auto', cmap='viridis_r',
                 extent=[resilience[0], resilience[-1], branching[0], branching[-1]])
plt.xlabel(r"resilience speed $\rho$ (1/s)")
plt.ylabel(r"branching ratio $\alpha/\beta$")
cbar = plt.colorbar(sns); cbar.set_label('cost')
plt.tight_layout()
plt.savefig("../docs/figures/sensitivity_heatmap.pdf")
plt.show()
# %% 5. table
sens.to_csv("../docs/tables/sensitivity_grid.csv", index=False)
sens.groupby('branching').mean().to_latex("../docs/tables/sensitivity_branching.tex")
sens.groupby('rho').mean().to_latex("../docs/tables/sensitivity_rho.tex")